In [1]:
%pip install semantic-link-labs

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 35.3 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-64b962e0-e00e-473d-8385-3725eee2795b
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Not uninstalling semantic-link-sempy at /home/trusted-se

In [2]:
import sempy_labs
import sempy.fabric as fabric  
from sempy_labs._helper_functions import resolve_workspace_name_and_id  
  
# Get both name and ID of current workspace  
workspace_name, workspace_id = resolve_workspace_name_and_id()
# List all connections  
connections_df = sempy_labs.list_connections()
#display(connections_df)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 9, Finished, Available, Finished)

#### Check connections to help verify which connections can be reused

In [3]:
# Define the list of connection types to keep
connection_types_to_keep = ["AzureDatabricksWorkspace", "AzureDataLakeStorage"]

# Filter the DataFrame for the desired connection types
filtered_connections_df = connections_df[connections_df["Connection Type"].isin(connection_types_to_keep)]

# Display the filtered DataFrame
display(filtered_connections_df)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4f1044f2-eeaa-4aae-9881-091e770dec44)

#### Check for existing Databricks mirrored items

In [4]:
item_list_df = fabric.list_items(workspace= workspace_id)
#display(item_list_df)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 11, Finished, Available, Finished)

In [5]:
# Define the list of connection types to keep
items_types_to_keep = ["MirroredAzureDatabricksCatalog"]

# Filter the DataFrame for the desired connection types
item_list_filtered = item_list_df[item_list_df["Type"].isin(items_types_to_keep)]

# Display the filtered DataFrame
display(item_list_filtered)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bb25258e-c893-4482-81b5-b5be5254abe7)

In [7]:

item_name_nameonly = item_list_filtered["Display Name"]
#display(item_name_nameonly)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 14, Finished, Available, Finished)

#### Get Databricks mirror item definition

In [8]:
from sempy_labs._helper_functions import get_item_definition  

def get_databricks_mirror_definition(item_name, workspace_name):  
    
    definition = get_item_definition(  
        item=item_name,  
        type="MirroredAzureDatabricksCatalog", 
        workspace= workspace_name,  
        return_dataframe=False,  
        decode=True  
    )
    print(f'###################### {item_name} ########################')
    print(definition)


StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 15, Finished, Available, Finished)

In [9]:
# Print out item definitions
for item_name in item_name_nameonly:
    get_databricks_mirror_definition(item_name, workspace_name)

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 16, Finished, Available, Finished)

###################### databricks279wus3_test1 ########################
{'definition': {'parts': [{'path': 'definition.json', 'payload': {'$schema': 'https://developer.microsoft.com/json-schemas/fabric/item/mirroredAzureDatabricksCatalog/definition/mirroredAzureDatabricksCatalogDefinition/1.0.0/schema.json', 'autoSync': 'Enabled', 'catalogName': 'databricks279wus3', 'databricksWorkspaceConnectionId': '469a508f-77d4-4a5f-a629-251fd0d3b4bf', 'mirrorConfiguration': {'schemas': [{'mirroringMode': 'Exclude', 'name': 'default'}, {'mirroringMode': 'Exclude', 'name': 'healthv'}]}, 'mirroringMode': 'Full'}}, {'path': '.platform', 'payload': {'$schema': 'https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json', 'metadata': {'type': 'MirroredAzureDatabricksCatalog', 'displayName': 'databricks279wus3_test1', 'description': ''}, 'config': {'version': '2.0', 'logicalId': '00000000-0000-0000-0000-000000000000'}}}]}}
###################### databricks279w

#### What can go wrong?


###### Azure Key Vault

If Firewall is set to Public Access Disabled, you will get an error:
- _IOException: 403 {"error":{"code":"Forbidden","message":"Connection is not an approved private link and caller was ignored because bypass is not set to 'AzureServices' and PublicNetworkAccess is set to 'Disabled'_
- _IOException: 403 {"error":{"code":"Forbidden","message":"Public network access is disabled and request is not from a trusted service nor via an approved private link._





In [10]:
fabric_ws_akv = "https://fab279akv1.vault.azure.net/"
adb_ws_id_s = "databricks-ws-id"
adb_ws_token_s= "databricks-ws-token"

# get the Databricks workspace id and PAT
dbx_workspace = notebookutils.credentials.getSecret(
    fabric_ws_akv,
    adb_ws_id_s)

print(f'Databricks workspace name is redacted: {dbx_workspace}')

StatementMeta(, 52bc70b4-928e-403c-9911-de5541099f7e, 17, Finished, Available, Finished)

Databricks workspace name is redacted: [REDACTED]
